### Módulos utilizados

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20,10)

<h1 style='color:purple'><b>1. Data Cleaning</b></h1>

### Cargo dataset

In [29]:
df1 = pd.read_csv('Bengaluru_House_Data.csv')
print(df1.shape)
df1.head(3)

(13320, 9)


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00


In [57]:
# groupby -> agrupa data points (rows) en las columnas indicadas
df1.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

### Elimino columnas irrelevantes

In [31]:
df2 = df1.drop(['area_type', 'society', 'balcony', 'availability'], axis='columns')
df2.head(3)

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00


### Elimino valores nulos mediante 

In [32]:
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [33]:
df3 = df2.dropna()
df3.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

### Analisis columna 'size'
'4 Bedroom' es lo mismo que '4 BHK'

In [34]:
# valores únicos de la columna 'size'

df3['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [35]:
# crear nueva columna ('bhk') a partir de la funcion lambda aplicada a 'size'
# x.split(' ')[0] -> separar cada vez que haya un espacio y tomar el primer término

df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

C:\Users\titi2\AppData\Local\Temp\ipykernel_6500\283496552.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


In [36]:
df3.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [37]:
df3['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [38]:
# como en el anterior bloque identifiqué propiedades con 27 y 43 dormitorios,
# quiero ver cuántas propiedades con más de 20 dormitorios hay en el dataset

df3[df3.bhk>20]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [39]:
df3.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [40]:
# qué clase de variaciones tengo en la columna 'total_sqft'?

# prueba a convertir el valor en float,
# si no puede retorna False, de lo contrario retorna True

def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [41]:
# observar que el tamaño de la propiedad en square feet
# a veces aparece como un rango, lo que es incorrecto

# ~ negador
df3[~df3['total_sqft'].apply(is_float)].head(10)

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4


In [42]:
# reemplazar rangos por sus promedios

def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2: # si 'tokens' es una lista de 2 elementos
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        return float(x)
    except:
        return None   

In [43]:
# probar función

convert_sqft_to_num('2100 - 2850')

2475.0

In [44]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)

# comparar con un promedio conocido
df4.loc[30]

location      Yelahanka
size              4 BHK
total_sqft       2475.0
bath                4.0
price             186.0
bhk                   4
Name: 30, dtype: object

<h1 style='color:purple'><b>2. Feature Engineering</b></h1>

In [45]:
# crear nueva columna de 'precio por unidad cuadrada' 

df5 = df4.copy()
df5['price_per_sqft'] = df5['price'] * 100000 / df5['total_sqft']
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [46]:
# contar canitdad de ubicaciones únicas
len(df5.location.unique())

1304

In [47]:
# tener tantas ubicaciones distintas nos impide hacer OneHotEncoding
# transformar strings a variables numéricas y luego dummy
# daria como resultado un dataframe de 1310 columnas

# cuántas filas tiene cada una de esas 1304 ubicaciones?
# de ser pocas, se pueden eliminar

# strip() -> elimina espacios al principio y al final de un string
df5.location = df5.location.apply(lambda x: x.strip())

location_stats = df5['location'].value_counts(ascending=False)
print(location_stats.head())

location_stats = df5['location'].value_counts(ascending=True)
print(location_stats.head())

# observar que hay muchas ubicaciones con un solo data-point (fila)

location
Whitefield         535
Sarjapur  Road     392
Electronic City    304
Kanakpura Road     266
Thanisandra        236
Name: count, dtype: int64
location
Abshot Layout                      1
Kanakadasa Layout                  1
Kengeri Satellite Town Stage II    1
Kasthuri Nagar East Of NGEF        1
Laxminarayana Layout               1
Name: count, dtype: int64


In [48]:
# mostrar cuántas ubicaciones tienen menos de 10 data-points

len(location_stats[location_stats <= 10])

1052

In [49]:
location_stats_less_than_10 = location_stats[location_stats <= 10]
location_stats_less_than_10

location
Abshot Layout                       1
Kanakadasa Layout                   1
Kengeri Satellite Town Stage II     1
Kasthuri Nagar East Of NGEF         1
Laxminarayana Layout                1
                                   ..
Gunjur Palya                       10
BTM 1st Stage                      10
Basapura                           10
1st Block Koramangala              10
Ganga Nagar                        10
Name: count, Length: 1052, dtype: int64

In [50]:
len(df5.location.unique())

1293

In [51]:
# eliminar ubicaciones con menos de 10 data points

df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

# ahora nos quedan 242 ubicaciones,
# con lo cual ya es viable hacer OneHotEncoding

242

<h1 style='color:purple'><b>3. Outlier Removal </b></h1>

In [52]:
# considerar casas con habitaciones de menos de 300 sqft como outliers

df5[df5.total_sqft / df5.bhk < 300].head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
9,other,6 Bedroom,1020.0,6.0,370.0,6,36274.509804
45,HSR Layout,8 Bedroom,600.0,9.0,200.0,8,33333.333333
58,Murugeshpalya,6 Bedroom,1407.0,4.0,150.0,6,10660.980810
68,Devarachikkanahalli,8 Bedroom,1350.0,7.0,85.0,8,6296.296296
70,other,3 Bedroom,500.0,3.0,100.0,3,20000.000000


In [53]:
df5.shape

(13246, 7)

In [54]:
# ~ negador
df6 = df5[~(df5.total_sqft / df5.bhk < 300)]
df6.shape

(12502, 7)

In [55]:
# considerar que 'price_per_sqft' es barato a  y caro a 

df6.price_per_sqft.describe()

count     12456.000000
mean       6308.502826
std        4168.127339
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

Here we find that min price per sqft is 267 rs/sqft whereas max is 12000000, this shows a wide variation in property prices. We should remove outliers per location using mean and one standard deviation

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [ ]:
df7 = remove_pps_outliers(df6)

print(df6.shape)
print(df7.shape)